## Security Management In the Cloud: Security Group Rules

We look at how security for instances inside the Cloud can be managed via OpenStack's Security Group 

### Task 1:

- Deploy an instance inside OpenStack using Ubuntu 12.04 with a floating IP address
- Setup an FTP server inside this instance
- Deploy a second instance inside OpenStack using the default `trusty-server` image and a floating IP address

In [1]:
# import relevant libraries
import geni.portal as portal
import geni.rspec.pg as rspec
import geni.rspec.igext as IG

# variable to define parameters related to the CloudLab web portal
pc = portal.Context()

# variable to define parameters related to the profile itself
request = rspec.Request()

# Generate the nodes
for i in range(4):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    node.routable_control_ip = True
    
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get -y update"))  
    
    #Enable NTP
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get -y install ntp"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo /etc/init.d/ntp start"))

    #Disable firewall
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo service ufw stop"))

    #Setup repositories
      
    ambariRepo = "http://public-repo-1.hortonworks.com/ambari/ubuntu14/2.x/updates/2.5.0.3/ambari.list"
    getEnvFile = "sudo wget " + ambariRepo + " -O /etc/apt/sources.list.d/ambari.list"
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command=getEnvFile)) 
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-key adv --recv-keys --keyserver keyserver.ubuntu.com B9733A7A07513CAD"))  
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get -y update"))  
        
    if i == 0:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo apt-get -y install ambari-server"))
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo ambari-server setup -s"))       
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo ambari-server start")) 
    else:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo apt-get -y install ambari-agent"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec(request)

<rspec xmlns:client="http://www.protogeni.net/resources/rspec/ext/client/1" xmlns:emulab="http://www.protogeni.net/resources/rspec/ext/emulab/1" xmlns:jacks="http://www.protogeni.net/resources/rspec/ext/jacks/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.geni.net/resources/rspec/3" xsi:schemaLocation="http://www.geni.net/resources/rspec/3 http://www.geni.net/resources/rspec/3/request.xsd" type="request">
  <rspec_tour xmlns="http://www.protogeni.net/resources/rspec/ext/apt-tour/1">
    <description type="markdown">Automatically created module for IPython interactive environment</description>
  </rspec_tour>
  <node client_id="node0" exclusive="true">
    <sliver_type name="raw">
      <disk_image name="urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"/>
    </sliver_type>
    <services>
      <execute shell="/bin/sh" command="sudo apt-get -y update"/>
      <execute shell="/bin/sh" command="sudo apt-get -y install ntp"/>
      <execute shell="/b

In [5]:
import geni.portal as portal
import geni.rspec.pg as rspec
import geni.rspec.igext as IG

pc = portal.Context()
request = rspec.Request()

pc.defineParameter('workerCount',
                   'Number of HPCC thorslaves (multiple of 4)',
                   portal.ParameterType.INTEGER, 4)

pc.defineParameter("controllerHost", "Name of controller node",
                   portal.ParameterType.STRING, "node0", advanced=True,
                   longDescription="The short name of the controller node.  You shold leave this alone unless you really want the hostname to change.")

params = pc.bindParameters()

tourDescription = "This profile provides a configurable HPCCSystems testbed one master and customizable thorslaves (multiples of four)."

tourInstructions = \
  """
### Basic Instructions
Once your experiment nodes have booted, and this profile's configuration scripts have finished deploying HPCCSystems inside your experiment, you'll be able to visit [the ECLWatch interface](http://{host-%s}:8010) (approx. 5-15 minutes).  
""" % (params.controllerHost)

#
# Setup the Tour info with the above description and instructions.
#  
tour = IG.Tour()
tour.Description(IG.Tour.TEXT,tourDescription)
tour.Instructions(IG.Tour.MARKDOWN,tourInstructions)
request.addTour(tour)


# Create a Request object to start building the RSpec.
#request = portal.context.makeRequestRSpec()
#request 
# Create a link with type LAN
link = request.LAN("lan")

# Generate the nodes
for i in range(params.workerCount + 1):
    node = request.RawPC("node" + str(i))
    node.disk_image = "urn:publicid:IDN+emulab.net+image+emulab-ops:UBUNTU14-64-STD"
    iface = node.addInterface("if" + str(i))
    iface.component_id = "eth1"
    iface.addAddress(rspec.IPv4Address("192.168.1." + str(i + 1), "255.255.255.0"))
    link.addInterface(iface)
    
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo adduser --ingroup admin --disabled-password hpcc"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo adduser hpcc sudo"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get update"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo wget http://cdn.hpccsystems.com/releases/CE-Candidate-5.2.2/bin/platform/hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo dpkg -i hpccsystems-platform-community_5.2.2-1trusty_amd64.deb"))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo apt-get -y -f install"))
    getEnvFile = "sudo wget https://raw.githubusercontent.com/clemsonbds/hpccsystems/master/environments/" + str(params.workerCount) + ".xml -O /etc/HPCCSystems/environment.xml"
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command=getEnvFile))
    node.addService(rspec.Execute(shell="/bin/sh",
                                  command="sudo wget https://raw.githubusercontent.com/clemsonbds/hpccsystems/master/conf/environment.conf -O /etc/HPCCSystems/environment.conf"))
    if i == 0:
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sleep 60"))
        node.addService(rspec.Execute(shell="/bin/sh",
                                      command="sudo service hpcc-init start"))

# Print the RSpec to the enclosing page.
portal.context.printRequestRSpec(request)

ArgumentError: argument --workerCount: conflicting option string(s): --workerCount

In [2]:
Create a new profile and copy/paste the above XML content into the `Edit Code` box

1;2
